In [47]:
import pandas as pd
import numpy as np
import openpyxl
from openpyxl.styles import PatternFill
from openpyxl.styles import Border, Side,Font
from openpyxl.utils.dataframe import dataframe_to_rows

In [48]:
def get_file_location():
    excel_file_path=r'C:\Users\anaso\Google Drive\Indiana_Master_of_Data_Science\Year3_Fall\Data Science In Practice\MM/Mentoring Grid FY23 - Pairing Results for DSIP - empty.xlsx'
    input_file_path=r'C:\Users\anaso\Google Drive\Indiana_Master_of_Data_Science\Year3_Fall\Data Science In Practice\MM/Mentoring and Matching Test File_anas (1).xlsx'
    sheet_name = 'Mentoring Grid FY23'
    return excel_file_path,input_file_path,sheet_name

In [49]:
def get_skills(excel_file_path,input_file_path):
    excel_file_path=excel_file_path
    input_file_path=input_file_path
    def split_choices_to_dict(choices):
        # Replace non-breaking spaces with regular spaces
        choices = choices.replace('\xa0', ' ')
        # Split by comma, then by newline, and strip spaces from each choice
        split_by_comma = choices.split(',')
        split_cleaned = [choice.strip() for c in split_by_comma for choice in c.split('\n') if choice.strip()]
        return {choice: int(index + 1) for index, choice in enumerate(split_cleaned)}

    
    df1=pd.read_excel(input_file_path)

    df2=pd.read_excel(excel_file_path,skiprows=1)
    #df2 = df2.loc[:, ~df2.columns.str.contains('^Unnamed')]


    # First, let's ensure that the 'Date' column in df2 is in datetime format for consistency
    df2['Date'] = pd.to_datetime(df2['Date'], errors='coerce')

    # Now let's create the loop to insert values into df2
    for index, row in df1.iterrows():
        # Create a new row dictionary starting with non-Mchoices data
        new_row = {
            'Application Type': row['Application Type'],
            'Submissions': row['Full Name'],
            'Email': row['Email Address'],
            'Date': pd.to_datetime(row['SubmitDate'])  # Convert to datetime
        }

        # Process the 'Mchoices' value into a dictionary
        mchoices_dict = split_choices_to_dict(row['Mchoices']) if pd.notna(row['Mchoices']) else {}

        # Loop through the dictionary and update new_row with the corresponding values
        for key, value in mchoices_dict.items():
            if key in df2.columns:  # Check if the key is a column name in df2
                new_row[key] = int(value)

        # Append the new row to df2
        df2 = df2.append(new_row, ignore_index=True)

    # Let's display the last few rows of df2 to confirm the changes
    return df2


In [50]:

def write_excel_matching_grid(excel_file_pathh,sheet_namee,df2 ):
    excel_file_path=excel_file_pathh
    sheet_name = sheet_namee  # Replace with the name of the sheet you want to write to

    # Define the color mapping for cell formatting
    color_mapping = {
        1: "92D050",  # Green
        2: "00B0F0",  # Blue
        3: "FFC000",  # Orange
        4: "FFFF00",  # Yellow
        5: "C00000",  # Red
    }
    application_type_colors = {
        'Mentor': "92D050",  # Example color for 'Type1'
        'Mentee': "B08270",  # Example color for 'Type2'
        # Add more mappings as needed
    }
    non_numeric_columns = ['Application Type', 'Submissions','Email','Date']
    # Load the workbook
    wb = openpyxl.load_workbook(excel_file_path)

    #wb.save(excel_file_path)
    # Select the specified sheet or default to active sheet if not found
    if sheet_name in wb.sheetnames:
        sheet = wb[sheet_name]
    else:
        sheet = wb.active  # Default to the active sheet if the specified one doesn't exist

    sheet.delete_rows(3, sheet.max_row - 1)
        # Find the last non-empty row in the sheet
    wb.save(excel_file_path)
    #last_row = 3
    wb = openpyxl.load_workbook(excel_file_path)
    if sheet_name in wb.sheetnames:
        sheet = wb[sheet_name]
    else:
        sheet = wb.active
    # Read the header row from the Excel file to get the column names
    header_row = [cell.value for cell in sheet[2]]

    # Matching columns
    matching_columns = [col for col in header_row if col in df2.columns]

    # Filter df2 to only include matching columns
    df2_matching = df2[matching_columns]
    df2_matching = df2_matching.dropna(how='all')
    for col in df2_matching.columns:
        if col not in non_numeric_columns:
            df2_matching[col] = pd.to_numeric(df2_matching[col], errors='coerce').fillna(0).astype(int)
            df2_matching[col] = df2_matching[col].replace(0, '')

    df2_matching['Date'] = df2_matching['Date'].dt.strftime('%Y-%m')
    df2_matching = df2_matching.sort_values(by=['Date', 'Application Type'])

    # Append rows from df2_matching to the Excel sheet
    for r_idx, row in enumerate(dataframe_to_rows(df2_matching, index=False, header=False), start=3):
        for c_idx, value in enumerate(row, start=1):
            cell = sheet.cell(row=r_idx, column=c_idx, value=value)
            #print("r_idx=",r_idx,"c_idx=",c_idx,"value=",value)

            # Apply color formatting if the cell contains an integer
            if isinstance(value, int) and value in color_mapping:
                cell.fill = PatternFill(start_color=color_mapping[value], end_color=color_mapping[value], fill_type="solid")

    columns_to_color = ['Application Type', 'Submissions', 'Email', 'Date']
    col_indices = {col: idx + 1 for idx, col in enumerate(header_row) if col in columns_to_color}
    for row in sheet.iter_rows(min_row=2, max_row=sheet.max_row):
        app_type = row[col_indices['Application Type'] - 1].value  # Adjust index for 0-based
        if app_type in application_type_colors:
            color = application_type_colors[app_type]
            for col_name, col_idx in col_indices.items():
                cell = row[col_idx - 1]  # Adjust index for 0-based
                cell.fill = PatternFill(start_color=color, end_color=color, fill_type="solid")



    thin_border = Border(left=Side(style='thin'), 
                         right=Side(style='thin'), 
                         top=Side(style='thin'), 
                         bottom=Side(style='thin'))
    bold_font = Font(bold=True,size=14,name='Helvetica')
    # Apply the border to each cell in the sheet
    for row in sheet.iter_rows():
        for cell in row:
            cell.border = thin_border 
            cell.font = bold_font


    # Save the updated workbook
    wb.save(excel_file_path)
    
    

In [51]:
#second

def perform_matching(excel_file_path,sheet_name):
    # Load the Excel file into a DataFrame
    df = pd.read_excel(excel_file_path, sheet_name=sheet_name,skiprows=1)

    # Define the columns for skills and priorities based on the screenshots provided
    skill_columns = df.columns.tolist()[4:]

    # Split the DataFrame into mentors and mentees
    mentors = df[df['Application Type'] == 'Mentor']
    mentees = df[df['Application Type'] == 'Mentee']

    # Create the matches DataFrame with mentors as rows and mentees as columns
    matches_df = pd.DataFrame(index=mentors['Submissions'], columns=mentees['Submissions'])

    # Iterate through mentors and mentees to calculate match scores
    for mentor_index, mentor_row in mentors.iterrows():
        mentor_skills = {skill: mentor_row[skill] for skill in skill_columns if not pd.isna(mentor_row[skill])}

        for mentee_index, mentee_row in mentees.iterrows():
            mentee_skills = {skill: mentee_row[skill] for skill in skill_columns if not pd.isna(mentee_row[skill])}
            common_skills = set(mentor_skills).intersection(set(mentee_skills))

            # Calculate the matching score based on the order of skills
            match_score = sum(1 for skill in mentee_skills if skill in common_skills and mentee_skills[skill] == mentor_skills[skill])
            total_mentee_skills = len(mentee_skills)
            match_percentage = (match_score / total_mentee_skills) * 100 if total_mentee_skills else 0

            # Populate the matches DataFrame
            matches_df.at[mentor_row['Submissions'], mentee_row['Submissions']] = match_percentage

    # Convert match percentages to a friendly display format (e.g., 50% instead of 0.5)
    matches_df = matches_df.applymap(lambda x: f'{x}%' if pd.notna(x) else '')

    # Save the matches DataFrame to a new sheet in the workbook
    with pd.ExcelWriter(excel_file_path, engine='openpyxl', mode='a') as writer:
        matches_df.to_excel(writer, sheet_name='Matches')

    # Return the path for the updated workbook
    


In [52]:
def main():
    excel_file_path,input_file_path,sheet_name=get_file_location()
    write_excel_matching_grid(excel_file_path,sheet_name,get_skills(excel_file_path,input_file_path) )
    perform_matching(excel_file_path,sheet_name)



In [53]:
main()